In [110]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold

import ml_metrics as metrics

from os.path import join

from global_constants import *

from glob import glob

from collections import defaultdict

from sklearn.preprocessing import MultiLabelBinarizer

In [111]:
def get_output():
    y = pd.read_csv("processed_tags.csv")
    return np.array(y)

In [112]:
lines = np.array([' '.join(f.split()) for i in range(10000) 
                  for f in open("features_train_prepossesing/"
                                + str(i) 
                                + ".txt")])
unique_words = list(set([item for sublist in lines.ravel() for item in sublist.split()]))

In [113]:
count_vect = TfidfVectorizer(vocabulary = unique_words)
X = count_vect.fit_transform(lines)

In [114]:
X.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [115]:
X.shape

(10000, 7791)

In [109]:
sum(X[-1].toarray())

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [100]:
y = get_output()

In [101]:
y.shape

(10000, 92)

## Train model

In [5]:
folds = 5
skf = StratifiedKFold(n_splits=folds)
accuracy_log = []
for train_index, test_index in skf.split(X, y):
    print("SPLIT")
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Create a random forest Classifier. By convention, clf means 'Classifier'
    clf = RandomForestClassifier(n_jobs=-1, random_state=0)

    clf.fit(X_train, y_train)
    
    accuracy_log.append(clf.score(y_test, X_test))
    
    print('RF accuracy:', accuracy_log[-1])
    
print("Results")
print("The average accuracy Random Forest is {}".format(folds, np.mean(accuracy_log)))

In [ ]:
# metrics.mapk